In [41]:
using Statistics
function coh(s_xx::AbstractMatrix{Float64}, s_yy::AbstractMatrix{Float64}, s_xy::AbstractMatrix{ComplexF64})::Array{Float64}
    # Compute the numerator: absolute value of the mean of s_xy along the last dimension
    con_num = abs.(mean(s_xy, dims=ndims(s_xy)))

    # Compute the denominator: square root of the product of means of s_xx and s_yy along the last dimension
    con_den = sqrt.(mean(s_xx, dims=ndims(s_xx)) .* mean(s_yy, dims=ndims(s_yy)))

    # Calculate coherence as the element-wise division of numerator by denominator
    coh = con_num ./ con_den
    return coh
end

function tril_indices(n)
    pairs = Array{Tuple{Int,Int},1}(undef, n * (n - 1) ÷ 2)
    q = 1
    for x in 1:n
        for y in (x+1):n
            pairs[q] = (x, y)
            q += 1
        end
    end
    return pairs
end

tril_indices (generic function with 1 method)

In [16]:
n_epochs = 606
n_channels = 97
n_times = 1024
n_taps = 3
freqs = collect(14:1023)
n_freqs = length(freqs)
n_cycles = 7
sfreq = 2048
nfft = 2048
mt_bandwidth = 4

weights = Array{Float64,3}(undef, n_taps, n_freqs, n_times)
Ws = Matrix{Vector{ComplexF64}}(undef, n_taps, n_freqs)

normalization = 2 ./ sum(real(weights .* conj(weights)), dims=1);

pairs = tril_indices(n_channels)
n_pairs = length(pairs)

4656

In [4]:
e = 44
tfrs = Array{ComplexF64,5}(undef, e, n_channels, n_taps, n_freqs, n_times);

psd_per_epoch = Array{Float64,4}(undef, e, n_channels, n_freqs, n_times)
coherence = Array{Float64,4}(undef, e, n_channels, n_channels, n_freqs);


In [6]:
size(tfrs), size(psd_per_epoch), size(coherence)

((44, 97, 3, 1010, 1024), (44, 97, 1010, 1024), (44, 97, 97, 1010))

In [42]:
idx = 1
# Calculate the epoch index and pair index
epoch_idx = div(idx - 1, n_pairs) + 1
pair_idx = mod(idx - 1, n_pairs) + 1
x, y = pairs[pair_idx]
# println("Epoch: $epoch_idx, Pair: ($x, $y)")
# Now perform your operations
w_x = @view tfrs[epoch_idx, x, :, :, :]
w_y = @view tfrs[epoch_idx, y, :, :, :]
s_xy = sum(weights .* w_x .* conj(weights .* w_y), dims=1)  # sum over tapers
s_xy .= s_xy .* normalization

s_xx = @view psd_per_epoch[epoch_idx, x, :, :]
s_yy = @view psd_per_epoch[epoch_idx, y, :, :]
coh_value = coh(s_xx, s_yy, s_xy[1, :, :])

# Copy to symmetric position
coherence[epoch_idx, y, x, :] .= coh_value

1010-element view(::Array{Float64, 4}, 1, 2, 1, :) with eltype Float64:
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
   ⋮
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN

In [ ]:
weights = 3×1010×1024 Array{Float64, 3}
normalization = 1×1010×1024 Array{Float64, 3}
tfrs = 44×97×3×1010×1024 Array{ComplexF64, 5}
psd_per_epoch = 44×97×1010×1024 Array{Float64, 4}

LoadError: ParseError:
[90m# Error @ [0;0m]8;;file:///media/dan/Data/git/network_mining/julia_v_python_connectivity/In[27]#1:8\[90mIn[27]:1:8[0;0m]8;;\
weights[48;2;120;70;70m 3×1010×1024 Array{Float64, 3}:[0;0m
[90m#      └─────────────────────────────┘ ── [0;0m[91mextra tokens after end of expression[0;0m

In [ ]:
coherence = Array{Float64,4}(e, n_channels, n_channels, n_freqs)

for epoch_idx = 1:e
for x = 1:c
    for y = (x+1):c  # Skip diagonal and only calculate for lower triangle
            w_x = tfrs[epoch_idx, x, :, :, :]
            w_y = tfrs[epoch_idx, y, :, :, :]
            s_xy = sum(weights .* w_x .* conj(weights .* w_y), dims=1)  # sum over tapers

            s_xy = s_xy .* normalization

            s_xx = psd_per_epoch[epoch_idx, x, :, :]
            s_yy = psd_per_epoch[epoch_idx, y, :, :]

            coh_value = coh(s_xx, s_yy, s_xy[1, :, :])
            # coherence[epoch_idx, x, y, :] .= coh_value # Copy to symmetric position
            coherence[epoch_idx, y, x, :] .= coh_value
        end
    end
end

In [ ]:



for idx = 1:(e * n_channels)
    # Compute epoch_idx and c_idx from idx
    epoch_idx = div(idx - 1, n_channels) + 1
    c_idx = mod(idx - 1, n_channels) + 1

    # Perform the element-wise multiplication with broadcasting
    psd = weights .* tfr[epoch_idx, c_idx, :, :, :]

    # Square magnitude (complex conjugate multiplication)
    psd = psd .* conj(psd)

    # Sum across the first dimension (dims=1 in Julia)
    psd = sum(real(psd), dims=1)

    # Apply the normalization
    psd = psd .* coh_normalization

    # Update the psd_per_epoch array
    psd_per_epoch[epoch_idx, c_idx, :, :] .= psd[1, :, :]
end


In [91]:
pairs = Array{Tuple{Int,Int},1}(undef, c * (c - 1) ÷ 2)
i = 1
for x in 1:c
    for y in (x+1):c
        pairs[i] = (x, y)
        i += 1
    end
end

n_pairs = length(pairs)

# Total number of iterations
total_iterations = e * n_pairs

for idx in 1:total_iterations
    # Calculate the epoch index and pair index
    epoch_idx = div(idx - 1, n_pairs) + 1
    pair_idx = mod(idx - 1, n_pairs) + 1
    x, y = pairs[pair_idx]
    println("Epoch: $epoch_idx, Pair: ($x, $y)")
    # # Now perform your operations
    # w_x = tfrs[epoch_idx, x, :, :, :]
    # w_y = tfrs[epoch_idx, y, :, :, :]
    # s_xy = sum(weights .* w_x .* conj(weights .* w_y), dims=1)  # sum over tapers
    # s_xy = s_xy .* normalization

    # s_xx = psd_per_epoch[epoch_idx, x, :, :]
    # s_yy = psd_per_epoch[epoch_idx, y, :, :]
    # coh_value = coh(s_xx, s_yy, s_xy[1, :, :])

    # # Copy to symmetric position
    # coherence[epoch_idx, y, x, :] .= coh_value
    break
end

In [98]:
function tril_indices(n)
    pairs = Array{Tuple{Int,Int},1}(undef, n * (n - 1) ÷ 2)
    q = 1
    for x in 1:n
        for y in (x+1):n
            pairs[q] = (x, y)
            q += 1
        end
    end
    return pairs
end

tril_indices(6)

15-element Vector{Tuple{Int64, Int64}}:
 (1, 2)
 (1, 3)
 (1, 4)
 (1, 5)
 (1, 6)
 (2, 3)
 (2, 4)
 (2, 5)
 (2, 6)
 (3, 4)
 (3, 5)
 (3, 6)
 (4, 5)
 (4, 6)
 (5, 6)